In [ ]:
import time
import torch

from diffusers import StableDiffusionPipeline, DDIMScheduler

In [ ]:
def save_and_display(image, pred_path):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    image.save(pred_path + "/" + timestr + ".png")
    display(image)

In [ ]:
model_path =  "/home/ubuntu/model"
pred_path = "/home/ubuntu/predict"
token_name = "sksbeikkpic"
num_pred_steps = 75
guide = 7.5
ddim = True

if ddim:
    ddim_scheduler = DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=False,
        )

    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=ddim_scheduler,
        safety_checker=None).to("cuda")
else:
    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        safety_checker=None).to("cuda")    

In [ ]:
prompt = "colorful cinematic portrait of " + token_name + " person, armor, cyberpunk,background made of brain cells, back light, organic, art by greg rutkowski, ultrarealistic, leica 30mm"

generator = torch.Generator(device="cuda")
seed = generator.seed()
generator = generator.manual_seed(seed)

width = 512
height = 512
latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = "cuda"
).half()

image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=guide).images[0]
save_and_display(image, pred_path)

In [ ]:
# Make sure latents are fixed
image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=guide).images[0]
save_and_display(image, pred_path)

In [ ]:
# less steps
image = pipe(prompt, latents = latents, num_inference_steps=25, guidance_scale=guide).images[0]
save_and_display(image, pred_path)

In [ ]:
# more steps
image = pipe(prompt, latents = latents, num_inference_steps=150, guidance_scale=guide).images[0]
save_and_display(image, pred_path)

In [ ]:
# even more steps
image = pipe(prompt, latents = latents, num_inference_steps=300, guidance_scale=guide).images[0]
save_and_display(image, pred_path)

In [ ]:
# weaker guide
image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=2.5).images[0]
save_and_display(image, pred_path)

In [ ]:
# stronger guide
image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=10).images[0]
save_and_display(image, pred_path)

In [ ]:
# stronger guide + more steps
image = pipe(prompt, latents = latents, num_inference_steps=300, guidance_scale=10).images[0]
save_and_display(image, pred_path)

In [ ]:
# No ddim
pipe_2 = StableDiffusionPipeline.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,
    safety_checker=None).to("cuda")
image = pipe_2(prompt, latents = latents, num_inference_steps=300, guidance_scale=10).images[0]
save_and_display(image, pred_path)

In [ ]:
# Negative prompt
image = pipe(prompt, negative_prompt="glasses", latents = latents, num_inference_steps=300, guidance_scale=10).images[0]
save_and_display(image, pred_path)

In [ ]:
# repeat special token
prompt = "colorful cinematic portrait of " + token_name + " person, " + token_name + " person, armor, cyberpunk,background made of brain cells, back light, organic, art by greg rutkowski, ultrarealistic, leica 30mm"

image = pipe(prompt, latents = latents, num_inference_steps=300, guidance_scale=10).images[0]
save_and_display(image, pred_path)